# СОДЕРЖАНИЕ

- 0. (подготовка)
- 1. Задача
- 4. **РЕШЕНИЕ**
- 2. Концепт решения
- 3. Решение step-by-step (алгоритмическое)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('test_data.csv', sep=';')
df = df.applymap(str)
df

,id_key,id_user,phone,email
0,1,12345,89997776655,test@mail.ru
1,2,54321,87778885566,two@mail.ru
2,3,98765,87776664577,three@mail.ru
3,4,66678,87778885566,four@mail.ru
4,5,34567,84547895566,four@mail.ru
5,6,34567,89087545678,five@mail.ru


In [3]:
df.dtypes

id_key     object
id_user    object
phone      object
email      object
dtype: object

# Задача

Написать Функцию:
- input вида "phone = 87778885566"
- output: связанные данный вида (см. ниже)

In [4]:
pd.read_csv('87778885566.csv', sep=';')

,id_key,id_user,phone,email
0,2,54321,87778885566,two@mail.ru
1,4,66678,87778885566,four@mail.ru
2,5,34567,84547895566,four@mail.ru
3,6,34567,89087545678,five@mail.ru


# РЕШЕНИЕ

In [5]:
def connection_query(df, query_s):
    
    # приведение к алгоритмическому решению
    cols = df.columns.to_list()[1:]
    input_ = query_s  
    
    # выделяем из запроса колонку и значение
    col = input_.split(' = ')[0]
    val = input_.split(' = ')[1]
    
    # создаем строку запроса
    q=f'{col} == "{val}"'
    
    # берем из основного DF данные по id_key, удовлетворяющие запросу q
    # то есть находим связь 1-ого уровня
    s1 = []
    for i in df.query(q).id_key.tolist():
        s1.append(i)
        
    '''
    # далее в коде нам понадобится ссылаться на имена колонок, поэтому
    cols = df.columns.to_list()[1:]
    '''
    
    # для каждого элемента из списка id_key перого уровня (s1) осуществляем поиск
    # задача: сформировать список id_key 2-ого уровня (s2)
    
    ####################
    # напишем ф-цию формирования списка id_key i-ого уровня из списка (i-1)-ого уровня
    def get_down(take_from, save_to, columns):
        
        ## для i-ого элемента верхнего уровня будем искать значения колонки 1 (id_user)
        for i in take_from:
            
            ### берем i-ый элемент верхнего уровня
            el_ = i

            ### формируем запрос
            q = f'id_key == "{el_}"'

            ### сохраняем результат запроса
            r = df.query(q)
            
            ### колонки нашего DataFrame'а
            cols = columns

            ### для k-ой колонки
            for k in cols:

                ### сохраняем название колонки, с которой работаем
                col_ = k

                ### из результата возьмем все данные из k-ой искомой колонки
                ### и сразу проведем поиск связанных данных
                v = r[col_].iloc[0]

                ### формируем запрос на поиск связанных с k-ой колонкой данных в id_key
                q = f'{col_} == "{v}"'

                ### ищем связанные с этим id_user данные в id_key
                ### и сохраняем эти id_key в список 2-ого уровня
                for i in df.query(q).id_key.tolist():
                    save_to.append(i)
    ####################
    
    # написали ф-цию формирования списка id_key i-ого уровня из списка (i-1)-ого уровня
    # вернемся к задаче
    
    ####################    
    # задача: сформировать список id_key 2-ого уровня (s2)
    # выполним ее с помощью ф-ции get_down()
    # для нее нужны:
    # take_from - список с id_keys верхнего уровня, 
    # save_to   - пустой список для id_keys нижнего уровня, 
    # columns   - список колонок DF, по к-рым будет осущ. поиск связи
    
    # take_from == s1
    s2 = []  # save_to
    # columns == cols
    
    # формируем список 2-ого уровня
    get_down(take_from=s1, save_to=s2, columns=cols)
    # теперь у нас есть сформированный список 2-ого уровня (s2)
    
    # удаляем дупликаты
    s2 = list(dict.fromkeys(s2))

    # сортируем
    s2 = list(map(int, s2))
    s2.sort()
    s2 = list(map(str, s2))

    # получаем список тех id_key, который мы еще не проверили на связь с первоначальным запросом
    # т.е. мы смотрим связь 2-ого уровня
    s2 = list(set(s2) - set(s1))
    ####################
    
    ####################    
    # задача: сформировать список id_key 3-ого уровня (s3)
    # выполним ее с помощью ф-ции get_down(), для к-рой нужны:
    # take_from == s2  - список с id_keys верхнего уровня, 
    s3 = []  # save_to - пустой список для id_keys нижнего уровня, 
    # columns == cols  - список колонок DF, по к-рым будет осущ. поиск связи
    
    # формируем список 3-его уровня
    get_down(take_from=s2, save_to=s3, columns=cols)
    # теперь у нас есть сформированный список 3-его уровня (s3)
    
    # удаляем дупликаты
    s3 = list(dict.fromkeys(s3))

    # сортируем
    s3 = list(map(int, s3))
    s3.sort()
    s3 = list(map(str, s3))

    # получаем список тех id_key, который мы еще не проверили на связь с первоначальным запросом
    # т.е. мы смотрим связь 3-его уровня
    s3 = list(set(s3) - set(s2) - set(s1))
    ####################

    ####################    
    # задача: сформировать список id_key 4-ого (финального для DataFrame'ов такого типа) уровня (s4)
    # выполним ее с помощью ф-ции get_down(), для к-рой нужны:
    # take_from == s3  - список с id_keys верхнего уровня, 
    s4 = []  # save_to - пустой список для id_keys нижнего уровня, 
    # columns == cols  - список колонок DF, по к-рым будет осущ. поиск связи
    
    # формируем список 4-ого уровня
    get_down(take_from=s3, save_to=s4, columns=cols)
    # теперь у нас есть сформированный список 4-его уровня (s4)
    
    # удаляем дупликаты
    s4 = list(dict.fromkeys(s4))

    # сортируем
    s4 = list(map(int, s4))
    s4.sort()
    s4 = list(map(str, s4))

    # получаем список тех id_key, который мы еще не проверили на связь с первоначальным запросом
    # т.е. мы смотрим связь 4-ого уровня
    s4 = list(set(s4) - set(s3) - set(s2) - set(s1))
    ####################    
    
    # не должно остаться id_key, которые мы не проверили на связь
    if s4 == []:
        pass
    else:
        print('ERROR: похоже, есть связь 5-ого и более уровней, т.к. список s4 не пуст')
        '''и здесь мы обрываем выполнение функции'''
        return 'ERROR'
    
    # формируем финальный список id_keys, которые мы должны отобразить в output
    S = set()
    S.update(set(s1))
    S.update(set(s2))
    S.update(set(s3))

    # возвращаем DF с запросом в качестве output
    
    return df.query(f'id_key in {list(S)}')

In [6]:
'''test'''
res = connection_query(df=df, query_s="phone = 87778885566")
res

,id_key,id_user,phone,email
1,2,54321,87778885566,two@mail.ru
3,4,66678,87778885566,four@mail.ru
4,5,34567,84547895566,four@mail.ru
5,6,34567,89087545678,five@mail.ru


# Концепт решения

Важное допущение:
- значения в колонке id_key никогда не повторяются

1. скарливаем строку вида "<колонка> = <значение>"
2. парсим и сохраняем <колонку> и <значение>
3. Шаг 1: находим в наборе данных (DF) колонку <колонка> и по ней ищем значения, равные <значению>; сохраняем id_key этих записей
4. Шаг 2: берем i-ый сохраненный id_key, сохраненный в п.3; находим в данных его запись; берем значения из k-ой колонки этой записи; по этому значению в DF в соответствующей колонке ищем равные значения и сохраняем их id_key
5. Шаг 3: все наборы сохраненных id_key объединяем без повторений и сохраняем результирующий набор
6. выводим данные из DF с соответствующими id_key

# Решение step-by-step

In [7]:
# вводим основной запрос
input_ = 'phone = 87778885566'

In [8]:
# выделяем из запроса колонку и значение
col = input_.split(' = ')[0]
val = input_.split(' = ')[1]

In [9]:
'''test'''
print(col)
print(val)

phone
87778885566


In [10]:
# создаем строку запроса
q=f'{col} == "{val}"'

In [11]:
'''test'''
q

'phone == "87778885566"'

In [12]:
# берем из основного DF данные по id_key, удовлетворяющие запросу q
# то есть находим связь 1-ого уровня
s1 = []
for i in df.query(q).id_key.tolist():
    s1.append(i)

In [13]:
'''test'''
s1

['2', '4']

In [14]:
'''test'''
df.query(q)

,id_key,id_user,phone,email
1,2,54321,87778885566,two@mail.ru
3,4,66678,87778885566,four@mail.ru


In [15]:
# далее в коде нам понадобится ссылаться на имена колонок, поэтому
cols = df.columns.to_list()[1:]

In [16]:
'''test'''
cols

['id_user', 'phone', 'email']

In [17]:
# для каждого элемента из списка id_key перого уровня (s1) осуществляем поиск

***

In [18]:
## для 1-ого элемента s1 будем искать значения колонки 1 (id_user)

### берем 1-ый элемент
el_ = s1[0]

In [19]:
'''test'''
print(s1)
print(el_)

['2', '4']
2


In [20]:
### формируем запрос
q = f'id_key == "{el_}"'

In [21]:
'''test'''
q

'id_key == "2"'

In [22]:
### сохраняем результат запроса
r = df.query(q)

In [23]:
'''test'''
r

,id_key,id_user,phone,email
1,2,54321,87778885566,two@mail.ru


***

In [24]:
### из результата возьмем все данные из 1-ой искомой колонки
### и сразу проведем поиск связанных данных
v = r[cols[0]].iloc[0]

In [25]:
'''test id_user'''
v

'54321'

In [26]:
### формируем запрос на поиск связанных с этим id_user (1-ая кол.) данных в id_key
q = f'{cols[0]} == "{v}"'

In [27]:
'''test'''
q

'id_user == "54321"'

In [28]:
### ищем связанные с этим id_user данные в id_key
### и сохраняем эти id_key в список 2-ого уровня
s2 = []
for i in df.query(q).id_key.tolist():
    s2.append(i)

In [29]:
'''test'''
df.query(q)

,id_key,id_user,phone,email
1,2,54321,87778885566,two@mail.ru


In [30]:
'''test'''
s2

['2']

***

In [31]:
### теперь
### из результата (r) возьмем все данные из 2-ой искомой колонки
### и сразу проведем поиск связанных данных
v = r[cols[1]].iloc[0]

In [32]:
'''test id_user'''
v

'87778885566'

In [33]:
### формируем запрос на поиск связанных с этим phone (2-ая кол.) данных в id_key
q = f'{cols[1]} == "{v}"'

In [34]:
'''test'''
q

'phone == "87778885566"'

In [35]:
### ищем связанные с этим phone данные в id_key
### и сохраняем эти id_key в список 2-ого уровня
'''s2 = []    это нам в этой ячейке не нужно'''
for i in df.query(q).id_key.tolist():
    s2.append(i)

In [36]:
'''test'''
df.query(q)

,id_key,id_user,phone,email
1,2,54321,87778885566,two@mail.ru
3,4,66678,87778885566,four@mail.ru


In [37]:
'''test'''
s2

['2', '2', '4']

***

In [38]:
### теперь
### из результата (r) возьмем все данные из 3-ей искомой колонки
### и сразу проведем поиск связанных данных
v = r[cols[2]].iloc[0]

### формируем запрос на поиск связанных с этим email (3-я кол.) данных в id_key
q = f'{cols[2]} == "{v}"'

### ищем связанные с этим email данные в id_key
### и сохраняем эти id_key в список 2-ого уровня
'''s2 = []    это нам в этой ячейке не нужно'''
for i in df.query(q).id_key.tolist():
    s2.append(i)

In [39]:
'''test'''
print(v)
print(q)
print(df.query(q))
print(s2)

two@mail.ru
email == "two@mail.ru"
  id_key id_user        phone        email
1      2   54321  87778885566  two@mail.ru
['2', '2', '4', '2']


***

In [40]:
## для 2-ого элемента s1 будем искать значения колонки 1 (id_user)

### берем 2-ой элемент
el_ = s1[1]

### формируем запрос
q = f'id_key == "{el_}"'

### сохраняем результат запроса
r = df.query(q)

### из результата возьмем все данные из 1-ой искомой колонки
### и сразу проведем поиск связанных данных
v = r[cols[0]].iloc[0]

### формируем запрос на поиск связанных с этим id_user (1-ая кол.) данных в id_key
q = f'{cols[0]} == "{v}"'

### ищем связанные с этим id_user данные в id_key
### и сохраняем эти id_key в список 2-ого уровня
'''s2 = []    на этом этапе при 1-ой кол. тоже уже не нужна эта строка'''
for i in df.query(q).id_key.tolist():
    s2.append(i)

### теперь
### из результата (r) возьмем все данные из 2-ой искомой колонки
### и сразу проведем поиск связанных данных
v = r[cols[1]].iloc[0]

### формируем запрос на поиск связанных с этим phone (2-ая кол.) данных в id_key
q = f'{cols[1]} == "{v}"'

### ищем связанные с этим phone данные в id_key
### и сохраняем эти id_key в список 2-ого уровня
for i in df.query(q).id_key.tolist():
    s2.append(i)
    
### теперь
### из результата (r) возьмем все данные из 3-ей искомой колонки
### и сразу проведем поиск связанных данных
v = r[cols[2]].iloc[0]

### формируем запрос на поиск связанных с этим email (3-я кол.) данных в id_key
q = f'{cols[2]} == "{v}"'

### ищем связанные с этим email данные в id_key
### и сохраняем эти id_key в список 2-ого уровня
'''s2 = []    это нам в этой ячейке не нужно'''
for i in df.query(q).id_key.tolist():
    s2.append(i)

In [41]:
'''test'''
s2

['2', '2', '4', '2', '4', '2', '4', '4', '5']

***

In [42]:
# для последнего элемента из списка id_key перого уровня (s1)  поиск  осуществлен
# теперь у нас есть сформированный список 2-ого уровня (s2)

In [43]:
# удаляем дупликаты
s2 = list(dict.fromkeys(s2))

In [44]:
# сортируем
s2 = list(map(int, s2))
s2.sort()
s2 = list(map(str, s2))

In [45]:
'''test'''
s2

['2', '4', '5']

In [46]:
# получаем список тех id_key, который мы еще не проверили на связь с первоначальным запросом
# т.е. мы смотрим связь 2-ого уровня
s2 = list(set(s2) - set(s1))

In [47]:
'''test'''
s2

['5']

***

In [48]:
s3=[]  # формируем 3-ий уровень связи
## для i-ого элемента s2 будем искать значения колонки 1 (id_user)
for i in s2:
    
    ### берем i-ый элемент
    el_ = i

    ### формируем запрос
    q = f'id_key == "{el_}"'

    ### сохраняем результат запроса
    r = df.query(q)

    ### для k-ой колонки
    for k in cols:
        
        ### сохраняем название колонки, с которой работаем
        col_ = k
        
        ### из результата возьмем все данные из k-ой искомой колонки
        ### и сразу проведем поиск связанных данных
        v = r[col_].iloc[0]

        ### формируем запрос на поиск связанных с k-ой колонкой данных в id_key
        q = f'{col_} == "{v}"'

        ### ищем связанные с этим id_user данные в id_key
        ### и сохраняем эти id_key в список 2-ого уровня
        for i in df.query(q).id_key.tolist():
            s3.append(i)
    
### теперь у нас сформирован список id_key, связанных с запросом на 3-ем уровне

In [49]:
# для последнего элемента из списка id_key 2-ого уровня (s2)  поиск  осуществлен
# теперь у нас есть сформированный список 3-его уровня (s3)

In [50]:
'''test'''
s3

['5', '6', '5', '4', '5']

In [51]:
# удаляем дупликаты
s3 = list(dict.fromkeys(s3))

# сортируем
s3 = list(map(int, s3))
s3.sort()
s3 = list(map(str, s3))

In [52]:
'''test'''
s3

['4', '5', '6']

In [53]:
# получаем список тех id_key, который мы еще не проверили на связь с первоначальным запросом
# т.е. мы смотрим связь 3-ого уровня
s3 = list(set(s3) - set(s2) - set(s1))

In [54]:
'''test'''
print(s1)
print(s2)

['2', '4']
['5']


In [55]:
'''test'''
s3

['6']

***

In [56]:
s4=[]  # формируем 4-ый (финальный для этого DF) уровень связи
## для i-ого элемента s3 будем искать значения колонки 1 (id_user)
for i in s3:
    
    ### берем i-ый элемент
    el_ = i

    ### формируем запрос
    q = f'id_key == "{el_}"'

    ### сохраняем результат запроса
    r = df.query(q)

    ### для k-ой колонки
    for k in cols:
        
        ### сохраняем название колонки, с которой работаем
        col_ = k
        
        ### из результата возьмем все данные из k-ой искомой колонки
        ### и сразу проведем поиск связанных данных
        v = r[col_].iloc[0]

        ### формируем запрос на поиск связанных с k-ой колонкой данных в id_key
        q = f'{col_} == "{v}"'

        ### ищем связанные с этим id_user данные в id_key
        ### и сохраняем эти id_key в список 2-ого уровня
        for i in df.query(q).id_key.tolist():
            s4.append(i)
    
### теперь у нас сформирован список id_key, связанных с запросом на 3-ем уровне

# для последнего элемента из списка id_key 3-ого уровня (s3)  поиск  осуществлен
# теперь у нас есть сформированный список 4-ого (финального для этого DF) уровня (s4)

# удаляем дупликаты
s4 = list(dict.fromkeys(s4))

# сортируем
s3 = list(map(int, s3))
s3.sort()
s3 = list(map(str, s3))

# получаем список тех id_key, который мы еще не проверили на связь с первоначальным запросом
# т.е. мы смотрим связь 4-ого уровня
s4 = list(set(s4) - set(s3) - set(s2) - set(s1))

In [57]:
'''test'''
s4

[]

In [58]:
# не должно остаться id_key, которые мы не проверили на связь
if s4 == []:
    pass
else:
    print('ERROR: похоже, есть связь 5-ого и более уровней, т.к. список s4 не пуст')
    '''и здесь мы обрываем выполнение функции'''

In [59]:
# формируем финальный список id_keys, которые мы должны отобразить в output
S = set()
S.update(set(s1))
S.update(set(s2))
S.update(set(s3))

In [60]:
'''test'''
S

{'2', '4', '5', '6'}

In [61]:
# возвращаем DF с запросом в качестве output
df.query(f'id_key in {list(S)}')

,id_key,id_user,phone,email
1,2,54321,87778885566,two@mail.ru
3,4,66678,87778885566,four@mail.ru
4,5,34567,84547895566,four@mail.ru
5,6,34567,89087545678,five@mail.ru
